<a href="https://www.kaggle.com/code/dimitar0dimov/knn-classifier-for-london-housing-average-prices?scriptVersionId=102584166" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np 
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data_monthly = pd.read_csv('/kaggle/input/housing-in-london/housing_in_london_monthly_variables.csv')
data_yearly = pd.read_csv('/kaggle/input/housing-in-london/housing_in_london_yearly_variables.csv')

data_monthly["date"] = pd.to_datetime(data_monthly["date"])
data_yearly["date"] = pd.to_datetime(data_yearly["date"])

In [ ]:
data_yearly.info()

Drop the emptiest column life_satisfaction as well as borough_flag because the latter is just ones and zeroes.

In [ ]:
data_yearly = data_yearly.drop(["life_satisfaction", "borough_flag"], axis=1)
data_yearly.isnull().sum()

Let's drop the rows with zeroes for specific columns first and then the remaining.

In [ ]:
data_yearly = data_yearly.dropna(subset=["median_salary","population_size","number_of_jobs"],how="any")
data_yearly.isnull().sum()

In [ ]:
data_yearly = data_yearly.dropna()
data_yearly.info()

In [ ]:
data_yearly.head(10)

Let's pre-process the monthly dataset and combine it with yearly so we can use all of the features to predict average prices.

In [ ]:
data_monthly.info()

In [ ]:
data_monthly.isnull().sum()

In [ ]:
data_monthly = data_monthly.dropna(subset=["houses_sold"],how="any")
data_monthly = data_monthly.drop(["no_of_crimes","borough_flag"],axis=1)
data_monthly.isnull().sum()

In [ ]:
data_monthly["year"] = pd.DatetimeIndex(data_monthly["date"]).year
data_monthly["month"] = pd.DatetimeIndex(data_monthly["date"]).month
data_monthly["day"] = pd.DatetimeIndex(data_monthly["date"]).day
data_monthly = data_monthly.drop(["date"],axis=1)

As we can see, the monthly dataset is much more granular than the yearly. We need to convert the monthly dataset into a yearly so we can join the two "yearly" datasets afterwords. 

We will take the average values for both average_price and houses_sold. 

In [ ]:
df3 = data_monthly.groupby(['year','code'])["average_price"].mean().reset_index()
df4 = data_monthly.groupby(['year','code'])["houses_sold"].mean().reset_index()
data_monthly_s = pd.merge(df3,df4,on=["year","code"])

In [ ]:
data_yearly["year"] = pd.DatetimeIndex(data_yearly["date"]).year
data_yearly["month"] = pd.DatetimeIndex(data_yearly["date"]).month
data_yearly["day"] = pd.DatetimeIndex(data_yearly["date"]).day
data_yearly = data_yearly.drop(["date"], axis=1)

Ready to merge the two datasets and apply the ML models.

In [ ]:
combined = pd.merge(data_yearly,data_monthly_s,on=["year","code"])

In [ ]:
combined.head(10)

In [ ]:
combined = combined.drop(["area","month","day"],axis=1)

Convert the string data entries from the code column into integers.

In [ ]:
from sklearn.preprocessing import LabelEncoder
LB = LabelEncoder()
combined["codeEnc"] = LB.fit_transform(combined["code"])
combined = combined.drop(["code"],axis=1)

I did this analysis on my on machine first and found out that there are some "#" entries in the mean_salary column. How? After I ran the first regression model it came back with an error "could not convert string to float". 

So I searched for an elegant solution to iterate through the dataframe and find where the "#" is. The following code was published on Stack Overflow. 


In [ ]:
mask = np.column_stack([combined[col].astype(str).str.contains(r"#", na=False) for col in combined])
combined.loc[mask.any(axis=1)]

In [ ]:
combined = combined[combined["mean_salary"] != '#']

I will try linear regression first and then knn classifier. 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics  
# Split df into X and y
y = combined['average_price']
X = combined.drop(['average_price'], axis=1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.65)

regressor = LinearRegression()  
regressor.fit(X_train, y_train) 

y_pred = regressor.predict(X_train) 
print('Mean Absolute Error (Train set):',metrics.mean_absolute_error(y_train, y_pred)) 
rmse = np.sqrt(np.mean((y_train - y_pred)**2))
print('Root Mean Absolute Error (Train set):',rmse)

In [ ]:
regressor.score(X_train, y_train)

In [ ]:
y_pred = regressor.predict(X_test) 
print('Mean Absolute Error (Test set):',metrics.mean_absolute_error(y_test, y_pred)) 
rmse = np.sqrt(np.mean((y_test - y_pred)**2))
print('Root Mean Sqaured Error (Test set):',rmse)

In [ ]:
regressor.score(X_test, y_test)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
knn_model = KNeighborsRegressor(n_neighbors=3)
knn_model.fit(X_train, y_train)
train_preds = knn_model.predict(X_train)
mse = mean_squared_error(y_train, train_preds)
rmse = sqrt(mse)
print('Root Mean Squared Error (Train set):',rmse)

In [ ]:
knn_model.score(X_train,y_train)

In [ ]:
test_preds = knn_model.predict(X_test)
mse = mean_squared_error(y_test, test_preds)
rmse = sqrt(mse)
print('Root Mean Squared Error (Test set):',rmse)

In [ ]:
knn_model.score(X_test,y_test)

Linear Regression score: **54.4%** \
KNeighbours Classifier score: **87.8%**

Let's see if scaling the values will improve the knn classifier at all.

In [ ]:
scaler = StandardScaler()
combined_sc = scaler.fit_transform(combined)
scaled_features_df = pd.DataFrame(combined_sc, index=combined.index, columns=combined.columns)
scaled_features_df

In [ ]:
y = scaled_features_df['average_price']
X = scaled_features_df.drop(['average_price'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

knn_model = KNeighborsRegressor(n_neighbors=3)
knn_model.fit(X_train, y_train)

test_preds = knn_model.predict(X_train)
mse = mean_squared_error(y_train, test_preds)
rmse = sqrt(mse)
rmse

In [ ]:
test_preds = knn_model.predict(X_test)
mse = mean_squared_error(y_test, test_preds)
rmse = sqrt(mse)
rmse

In [ ]:
knn_model.score(X_test,y_test)

Nope, not really. 